<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
# <font color='blue'>Data Science Academy</font>
## <font color='blue'>Armazenamento e Gestão de Dados com Data Lake e Data Lakehouse</font>
## <font color='blue'>Lab 6</font>
### <font color='blue'>Operações CRUD e Time Travel com Delta Lake</font>
### <font color='blue'>Exemplo 2</font>

In [1]:
# Imports
from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip
from delta.tables import DeltaTable
from pyspark.sql.functions import col, to_json, lit, collect_list, size

In [2]:
# Configuração do Spark com Delta Lake
builder = SparkSession.builder \
    .appName("Lab6Exemplo2") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.sql.debug.maxToStringFields", "5000")  

In [4]:
spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [5]:
# Define o nível de log
spark.sparkContext.setLogLevel("ERROR")

In [6]:
# Caminho da tabela Delta
delta_table_path = "/repositorio/dsa-delta-table-exemplo2"

## Executando as Operações em Sequência

In [7]:
# CREATE - Cria o dataframe com os dados iniciais
dsa_dados = spark.createDataFrame([
    (1, "Ana", 30, "Cientista de Dados"),
    (2, "Bob", 18, "Analista de Dados"),
    (3, "Charlie", 35, "Arquiteto de Dados"),
    (4, "Mark", 40, "Engenheiro de Dados"),
    (5, "Eve", 28, "Engenheiro de IA"),
    (6, "Frank", 50, "Engenheiro de ML"),
    (7, "Grace", 29, "Engenheiro DataOps"),
    (8, "Hank", 33, "Engenheiro de Dados")
], ["id", "nome", "idade", "funcao"])

In [8]:
# Gravar dados na tabela Delta
dsa_dados.write.format("delta").mode("overwrite").save(delta_table_path)

In [9]:
# READ - Ler os dados
print("Dados iniciais:")
spark.read.format("delta").load(delta_table_path).show()

Dados iniciais:


+---+-------+-----+-------------------+
| id|   nome|idade|             funcao|
+---+-------+-----+-------------------+
|  3|Charlie|   35| Arquiteto de Dados|
|  7|  Grace|   29| Engenheiro DataOps|
|  8|   Hank|   33|Engenheiro de Dados|
|  4|   Mark|   40|Engenheiro de Dados|
|  6|  Frank|   50|   Engenheiro de ML|
|  1|    Ana|   30| Cientista de Dados|
|  2|    Bob|   18|  Analista de Dados|
|  5|    Eve|   28|   Engenheiro de IA|
+---+-------+-----+-------------------+



In [10]:
# UPDATE - Atualizar idade e função de um funcionário
delta_table = DeltaTable.forPath(spark, delta_table_path)
delta_table.update(
    condition = "nome = 'Ana'",
    set = {"idade": "32", "funcao": "'Gerente de Data Science'"}
)
print("Após atualização de Ana:")
delta_table.toDF().show(truncate=False)

Após atualização de Ana:


[Stage 24:======================================================> (49 + 1) / 50]

+---+-------+-----+-----------------------+
|id |nome   |idade|funcao                 |
+---+-------+-----+-----------------------+
|1  |Ana    |32   |Gerente de Data Science|
|3  |Charlie|35   |Arquiteto de Dados     |
|7  |Grace  |29   |Engenheiro DataOps     |
|8  |Hank   |33   |Engenheiro de Dados    |
|4  |Mark   |40   |Engenheiro de Dados    |
|6  |Frank  |50   |Engenheiro de ML       |
|2  |Bob    |18   |Analista de Dados      |
|5  |Eve    |28   |Engenheiro de IA       |
+---+-------+-----+-----------------------+



In [11]:
# DELETE - Remover registros com idade menor ou igual a 18
delta_table.delete(condition = "idade <= 18")
print("Após remoção de funcionários com idade <= 18:")
delta_table.toDF().show(truncate=False)

Após remoção de funcionários com idade <= 18:


+---+-------+-----+-----------------------+
|id |nome   |idade|funcao                 |
+---+-------+-----+-----------------------+
|1  |Ana    |32   |Gerente de Data Science|
|3  |Charlie|35   |Arquiteto de Dados     |
|7  |Grace  |29   |Engenheiro DataOps     |
|8  |Hank   |33   |Engenheiro de Dados    |
|4  |Mark   |40   |Engenheiro de Dados    |
|6  |Frank  |50   |Engenheiro de ML       |
|5  |Eve    |28   |Engenheiro de IA       |
+---+-------+-----+-----------------------+



In [12]:
# INSERT - Inserir múltiplos novos registros
new_employees = spark.createDataFrame([
    (9, "Ivy", 27, "Analytics Engineer"),
    (10, "Jake", 31, "Analytics Engineer"),
    (11, "Liam", 26, "Engenheiro de Dados")
], ["id", "nome", "idade", "funcao"])

delta_table.alias("existingData").merge(
    new_employees.alias("newData"),
    "existingData.id = newData.id"
).whenNotMatchedInsertAll().execute()

print("Após inserção de novos registros:")
delta_table.toDF().show(truncate=False)

Após inserção de novos registros:


+---+-------+-----+-----------------------+
|id |nome   |idade|funcao                 |
+---+-------+-----+-----------------------+
|1  |Ana    |32   |Gerente de Data Science|
|3  |Charlie|35   |Arquiteto de Dados     |
|11 |Liam   |26   |Engenheiro de Dados    |
|7  |Grace  |29   |Engenheiro DataOps     |
|8  |Hank   |33   |Engenheiro de Dados    |
|4  |Mark   |40   |Engenheiro de Dados    |
|10 |Jake   |31   |Analytics Engineer     |
|9  |Ivy    |27   |Analytics Engineer     |
|6  |Frank  |50   |Engenheiro de ML       |
|5  |Eve    |28   |Engenheiro de IA       |
+---+-------+-----+-----------------------+



## UPSERT - Inserir ou Atualizar (Merge) Dados Existentes e Novos

In [13]:
# UPSERT - Inserir ou Atualizar (Merge) dados existentes e novos
upsert_data = spark.createDataFrame([
    (3, "Charlie", 36, "Arquiteto de Dados"),  # Atualizar idade de Charlie
    (12, "Maria", 24, "Arquiteto RPA")  # Novo registro
], ["id", "nome", "idade", "funcao"])

delta_table.alias("oldData").merge(
    upsert_data.alias("upsertData"),
    "oldData.id = upsertData.id"
).whenMatchedUpdate(set={
    "idade": "upsertData.idade",
    "funcao": "upsertData.funcao"
}).whenNotMatchedInsertAll().execute()

print("Após upsert (atualização/inserção):")
delta_table.toDF().show(truncate=False)

Após upsert (atualização/inserção):


+---+-------+-----+-----------------------+
|id |nome   |idade|funcao                 |
+---+-------+-----+-----------------------+
|1  |Ana    |32   |Gerente de Data Science|
|11 |Liam   |26   |Engenheiro de Dados    |
|7  |Grace  |29   |Engenheiro DataOps     |
|8  |Hank   |33   |Engenheiro de Dados    |
|4  |Mark   |40   |Engenheiro de Dados    |
|10 |Jake   |31   |Analytics Engineer     |
|9  |Ivy    |27   |Analytics Engineer     |
|6  |Frank  |50   |Engenheiro de ML       |
|3  |Charlie|36   |Arquiteto de Dados     |
|12 |Maria  |24   |Arquiteto RPA          |
|5  |Eve    |28   |Engenheiro de IA       |
+---+-------+-----+-----------------------+



## Filtros e Análises

In [14]:
# Filtrar funcionários com uma determinada função
print("Funcionários que são Engenheiros de Dados:")
delta_table.toDF().filter("funcao = 'Engenheiro de Dados'").show(truncate=False)

Funcionários que são Engenheiros de Dados:
+---+----+-----+-------------------+
|id |nome|idade|funcao             |
+---+----+-----+-------------------+
|11 |Liam|26   |Engenheiro de Dados|
|8  |Hank|33   |Engenheiro de Dados|
|4  |Mark|40   |Engenheiro de Dados|
+---+----+-----+-------------------+



In [15]:
# Agrupar por função e calcular média de idade
print("Média de idade por função:")
delta_table.toDF().groupBy("funcao").avg("idade").show(truncate=False)

Média de idade por função:
+-----------------------+----------+
|funcao                 |avg(idade)|
+-----------------------+----------+
|Gerente de Data Science|32.0      |
|Engenheiro de Dados    |33.0      |
|Engenheiro DataOps     |29.0      |
|Analytics Engineer     |29.0      |
|Engenheiro de ML       |50.0      |
|Engenheiro de IA       |28.0      |
|Arquiteto de Dados     |36.0      |
|Arquiteto RPA          |24.0      |
+-----------------------+----------+



In [16]:
# Agrupar por função, calcular a média de idade e filtrar por média > 30
print("Funções com média de idade maior que 30:")
delta_table.toDF() \
    .groupBy("funcao") \
    .avg("idade") \
    .filter(col("avg(idade)") > 30) \
    .show(truncate=False)

Funções com média de idade maior que 30:
+-----------------------+----------+
|funcao                 |avg(idade)|
+-----------------------+----------+
|Gerente de Data Science|32.0      |
|Engenheiro de Dados    |33.0      |
|Engenheiro de ML       |50.0      |
|Arquiteto de Dados     |36.0      |
+-----------------------+----------+



## Histórico de Alterações (Time Travel)

In [17]:
# Caminho para a tabela Delta
delta_table = DeltaTable.forPath(spark, delta_table_path)

# Obter o histórico completo
history_df = delta_table.history()

# Contar o número de versões
num_versions = history_df.count()
print(f"A tabela tem {num_versions} versões.")

A tabela tem 5 versões.


In [18]:
# Acessar a versão mais antiga da tabela (versão 0)
print("Versão inicial da tabela:")
old_version = spark.read.format("delta").option("versionAsOf", 0).load(delta_table_path)
old_version.show(truncate=False)

Versão inicial da tabela:


+---+-------+-----+-------------------+
|id |nome   |idade|funcao             |
+---+-------+-----+-------------------+
|3  |Charlie|35   |Arquiteto de Dados |
|7  |Grace  |29   |Engenheiro DataOps |
|8  |Hank   |33   |Engenheiro de Dados|
|4  |Mark   |40   |Engenheiro de Dados|
|6  |Frank  |50   |Engenheiro de ML   |
|1  |Ana    |30   |Cientista de Dados |
|2  |Bob    |18   |Analista de Dados  |
|5  |Eve    |28   |Engenheiro de IA   |
+---+-------+-----+-------------------+



In [19]:
print("Versão 0 da tabela:")
version_0 = spark.read.format("delta").option("versionAsOf", 0).load(delta_table_path)
version_0.show(truncate=False)

print("Versão 1 da tabela:")
version_1 = spark.read.format("delta").option("versionAsOf", 1).load(delta_table_path)
version_1.show(truncate=False)

print("Versão 4 da tabela:")
version_4 = spark.read.format("delta").option("versionAsOf", 4).load(delta_table_path)
version_4.show(truncate=False)

Versão 0 da tabela:


+---+-------+-----+-------------------+
|id |nome   |idade|funcao             |
+---+-------+-----+-------------------+
|3  |Charlie|35   |Arquiteto de Dados |
|7  |Grace  |29   |Engenheiro DataOps |
|8  |Hank   |33   |Engenheiro de Dados|
|4  |Mark   |40   |Engenheiro de Dados|
|6  |Frank  |50   |Engenheiro de ML   |
|1  |Ana    |30   |Cientista de Dados |
|2  |Bob    |18   |Analista de Dados  |
|5  |Eve    |28   |Engenheiro de IA   |
+---+-------+-----+-------------------+

Versão 1 da tabela:


+---+-------+-----+-----------------------+
|id |nome   |idade|funcao                 |
+---+-------+-----+-----------------------+
|1  |Ana    |32   |Gerente de Data Science|
|3  |Charlie|35   |Arquiteto de Dados     |
|7  |Grace  |29   |Engenheiro DataOps     |
|8  |Hank   |33   |Engenheiro de Dados    |
|4  |Mark   |40   |Engenheiro de Dados    |
|6  |Frank  |50   |Engenheiro de ML       |
|2  |Bob    |18   |Analista de Dados      |
|5  |Eve    |28   |Engenheiro de IA       |
+---+-------+-----+-----------------------+

Versão 4 da tabela:
+---+-------+-----+-----------------------+
|id |nome   |idade|funcao                 |
+---+-------+-----+-----------------------+
|1  |Ana    |32   |Gerente de Data Science|
|11 |Liam   |26   |Engenheiro de Dados    |
|7  |Grace  |29   |Engenheiro DataOps     |
|8  |Hank   |33   |Engenheiro de Dados    |
|4  |Mark   |40   |Engenheiro de Dados    |
|10 |Jake   |31   |Analytics Engineer     |
|9  |Ivy    |27   |Analytics Engineer     |
|6  |Frank 

In [20]:
# Adicionar uma coluna que identifica a versão
version_0 = version_0.withColumn("versao", lit(0))
version_4 = version_4.withColumn("versao", lit(4))

# Unir as duas versões
changes = version_0.union(version_4)

In [21]:
# Mostrar as diferenças em relação ao nome
print("Alterações entre versões:")
changes.groupBy("id", "nome") \
       .agg(collect_list("versao").alias("versoes")) \
       .filter(size("versoes") == 1) \
       .show(truncate=False)

Alterações entre versões:
+---+-----+-------+
|id |nome |versoes|
+---+-----+-------+
|2  |Bob  |[0]    |
|11 |Liam |[4]    |
|10 |Jake |[4]    |
|9  |Ivy  |[4]    |
|12 |Maria|[4]    |
+---+-----+-------+



In [22]:
# Mostrar as diferenças em relação a idade
print("Alterações entre versões:")
changes.groupBy("id", "idade") \
       .agg(collect_list("versao").alias("versoes")) \
       .filter(size("versoes") == 1) \
       .show(truncate=False)

Alterações entre versões:
+---+-----+-------+
|id |idade|versoes|
+---+-----+-------+
|3  |35   |[0]    |
|1  |30   |[0]    |
|2  |18   |[0]    |
|1  |32   |[4]    |
|11 |26   |[4]    |
|10 |31   |[4]    |
|9  |27   |[4]    |
|3  |36   |[4]    |
|12 |24   |[4]    |
+---+-----+-------+



In [23]:
# Carrega a tabela delta
delta_table = DeltaTable.forPath(spark, delta_table_path)

# Carregar o histórico de alterações da tabela Delta
history = delta_table.history()

# Selecionar apenas as colunas relevantes
formatted_history = history.select(
    col("version").alias("Versão"),
    col("operation").alias("Operação"),
    col("operationMetrics").alias("Métricas"),
    col("userMetadata").alias("Metadados do Usuário")
)

# Mostrar as alterações 
print("Histórico de alterações da tabela Delta (formatado):")
formatted_history.show(truncate=False)

Histórico de alterações da tabela Delta (formatado):
+------+--------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+
|Versão|Operação|Métricas                                                                                                                                                                                                                                                           

In [24]:
# Se você quiser exibir apenas operações específicas (por exemplo, UPDATE), pode usar .filter():
filtered_history = formatted_history.filter(col("Operação") == "UPDATE")
filtered_history.show(truncate=False)

+------+--------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+
|Versão|Operação|Métricas                                                                                                                                                                                                                                                                                                                      |Metadados do Usuário|
+------+--------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [25]:
# Carregar o histórico de alterações da tabela Delta
history = delta_table.history()

# Selecionar e formatar as colunas
formatted_history = history.select(
    col("version").alias("Versão"),
    col("operation").alias("Operação"),
    to_json(col("operationMetrics")).alias("Métricas"),  # Converter MAP para JSON, para conseguir salvar em CSV
    col("userMetadata").alias("Metadados do Usuário")
)

# Salvar o histórico formatado em CSV
output_path = "output"
formatted_history.write.format("csv").option("header", "true").save(output_path)

print(f"Histórico salvo em: {output_path}")

Histórico salvo em: output


Embora não haja um comando direto de rollback no Delta Lake, você pode sobrescrever uma nova versão com os dados de uma versão anterior sem perder todo o histórico:

In [26]:
# Consultar uma versão antiga (versão 2)
spark.read.format("delta").option("versionAsOf", 2).load(delta_table_path).show(truncate=False)

+---+-------+-----+-----------------------+
|id |nome   |idade|funcao                 |
+---+-------+-----+-----------------------+
|1  |Ana    |32   |Gerente de Data Science|
|3  |Charlie|35   |Arquiteto de Dados     |
|7  |Grace  |29   |Engenheiro DataOps     |
|8  |Hank   |33   |Engenheiro de Dados    |
|4  |Mark   |40   |Engenheiro de Dados    |
|6  |Frank  |50   |Engenheiro de ML       |
|5  |Eve    |28   |Engenheiro de IA       |
+---+-------+-----+-----------------------+



In [27]:
# Carregar a versão 2
old_version = spark.read.format("delta").option("versionAsOf", 2).load(delta_table_path)

In [28]:
# Sobrescrever a tabela principal com a versão 2
# Isso vai gerar uma cópia da versão 2 que será agora a versão principal. 
old_version.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save(delta_table_path)

In [29]:
# Verificar os dados sobrescritos
spark.read.format("delta").load(delta_table_path).show(truncate=False)

+---+-------+-----+-----------------------+
|id |nome   |idade|funcao                 |
+---+-------+-----+-----------------------+
|1  |Ana    |32   |Gerente de Data Science|
|3  |Charlie|35   |Arquiteto de Dados     |
|4  |Mark   |40   |Engenheiro de Dados    |
|8  |Hank   |33   |Engenheiro de Dados    |
|7  |Grace  |29   |Engenheiro DataOps     |
|6  |Frank  |50   |Engenheiro de ML       |
|5  |Eve    |28   |Engenheiro de IA       |
+---+-------+-----+-----------------------+



In [30]:
# Caminho para a tabela Delta
delta_table = DeltaTable.forPath(spark, delta_table_path)

# Obter o histórico completo
history_df = delta_table.history()

# Contar o número de versões
num_versions = history_df.count()
print(f"A tabela tem {num_versions} versões.")

A tabela tem 6 versões.


Considerações:

- A operação de sobrescrita cria uma nova versão na tabela Delta. Dados atuais ainda estarão no histórico, mas os dados sobrescritos substituem a visão principal da tabela.

- Certifique-se de que o esquema da versão antiga é compatível com o esquema atual. Caso contrário, pode ser necessário habilitar a opção overwriteSchema.

- Em ambientes críticos, prefira corrigir os dados com operações como MERGE ou UPDATE em vez de sobrescrever diretamente.

## VACUUM Com Valor Seguro Para Retenção

Por padrão, o Delta Lake impõe uma retenção mínima de 7 dias para garantir que operações como time travel ainda sejam possíveis e para evitar exclusão acidental de dados necessários para transações. Se você quiser reduzir esse período, será necessário modificar a configuração de retenção mínima.

Você não poderá acessar versões anteriores além do período de retenção configurado.

In [31]:
# Desativar temporariamente a proteção para retenção mínima
spark.sql("SET spark.databricks.delta.retentionDurationCheck.enabled = false")

DataFrame[key: string, value: string]

In [32]:
# Executar VACUUM com retenção de 1 dia
print("Executando vacuum com retenção de 1 dia...")
delta_table.vacuum(retentionHours=24)

Executando vacuum com retenção de 1 dia...


Deleted 0 files and directories in a total of 1 directories.


DataFrame[]

In [33]:
# Reativar a proteção para retenção mínima
spark.sql("SET spark.databricks.delta.retentionDurationCheck.enabled = true")

DataFrame[key: string, value: string]

Se o VACUUM foi executado com um período curto de retenção, versões mais antigas podem ter sido excluídas e não estarão disponíveis no histórico.


In [34]:
# Finaliza a sessão Spark
spark.stop()

# Fim